In [2]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import math 

PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'sarah-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

SEQ_LEN = 50

### Define the project, Bucket and Region and set them.

In [3]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [4]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [5]:
! pwd
! ls

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		 data		  trained
BIG_q.ipynb		 Energy_ts	  train_model_local.ipynb
build_local_model.ipynb  hyperparam.yaml  Untitled Notebook.ipynb
Build_model_task.ipynb	 sarah_data.csv
CNN_W2P_model		 test.json


In [7]:
df2 = pd.read_csv('data/CNN_W2P_model/train-1.csv', header = None)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.948333,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,...,0.658333,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667
1,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,...,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333
2,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,...,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500
3,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,...,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667
4,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,0.816667,...,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667,0.492167


In [8]:
df2 = pd.read_csv('data/CNN_W2P_model/valid-1.csv', header = None) #, names = CSV_COLUMNS)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.893333,0.550000,0.886500,1.018167,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,...,0.725000,0.983167,0.946167,0.919833,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333
1,0.550000,0.886500,1.018167,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,0.766000,...,0.983167,0.946167,0.919833,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333,0.744167
2,0.886500,1.018167,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,0.766000,1.000000,...,0.946167,0.919833,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333,0.744167,1.001667
3,1.018167,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,0.766000,1.000000,0.983333,...,0.919833,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333,0.744167,1.001667,0.993333
4,0.668667,0.863333,0.718333,0.960000,0.711500,0.805833,0.766000,1.000000,0.983333,0.970833,...,1.011500,1.116833,1.019833,0.920833,0.794833,1.083333,0.744167,1.001667,0.993333,0.865000


In [9]:
%%bash
rm -rf /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
mkdir /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
touch /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/__init__.py

In [10]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

TIMESERIES_COL = 'price'
N_OUTPUTS = 1  # in each sequence, 1-49 are features, and 50 is label
SEQ_LEN = None
DEFAULTS = None
N_INPUTS = None


def init(hparams):
    global SEQ_LEN, DEFAULTS, N_INPUTS
    SEQ_LEN = hparams['sequence_length']
    DEFAULTS = [[0.0] for x in range(0, SEQ_LEN)]
    N_INPUTS = SEQ_LEN - N_OUTPUTS


def linear_model(features, mode, params):
    X = features[TIMESERIES_COL]
    predictions = tf.layers.dense(X, 1, activation=None)
    return predictions


def dnn_model(features, mode, params):
    X = features[TIMESERIES_COL]
    h1 = tf.layers.dense(X, 10, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # linear output: regression
    return predictions


def cnn_model(features, mode, params):
    X = tf.reshape(features[TIMESERIES_COL],
                   [-1, N_INPUTS, 1])  # as a 1D "sequence" with only one time-series observation (height)
    c1 = tf.layers.conv1d(X, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p1 = tf.layers.max_pooling1d(c1, pool_size=2, strides=2)

#     outlen = p1.shape[1] * p1.shape[2]
#     c1flat = tf.reshape(p1, [-1, outlen])
#     h1 = tf.layers.dense(c1flat, 3, activation=tf.nn.relu)
#     predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
#     return predictions
    
    c2 = tf.layers.conv1d(p1, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p2 = tf.layers.max_pooling1d(c2, pool_size=2, strides=2)

    outlen = p2.shape[1] * p2.shape[2]
    c2flat = tf.reshape(p2, [-1, outlen])
    h1 = tf.layers.dense(c2flat, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
    return predictions


def rnn_model(features, mode, params):
    CELL_SIZE = N_INPUTS // 3  # size of the internal state in each of the cells

    # 1. dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell = tf.nn.rnn_cell.GRUCell(CELL_SIZE)
    outputs, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state, N_INPUTS // 2, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, N_INPUTS // 2, activation=tf.nn.relu6)
    h3 = tf.layers.dense(h2, N_INPUTS // 2, activation=tf.nn.relu)
    
    predictions = tf.layers.dense(h3, 1, activation=None)  # (?, 1)
    return predictions


# 2-layer RNN
def rnn2_model(features, mode, params):
    # 1. dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 3)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 3)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'state' is now a tuple containing the final state of each cell layer
    # we use state[1] below to extract the final state of the final layer
    outputs = outputs[:, (N_INPUTS-1):, :] # last one only
  
    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state[1], cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)
    return predictions


# create N-1 predictions
def rnnN_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'outputs' contains the state of the final layer for every time step
    # not just the last time step (?,N_INPUTS, final cell size)
    
    # 3. pass state for each time step through a DNN, to get a prediction
    # for each time step 
    h1 = tf.layers.dense(outputs, cells.output_size, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # (?, N_INPUTS, 1)
    predictions = tf.reshape(predictions, [-1, N_INPUTS])
    return predictions # return prediction for each time step


# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return {TIMESERIES_COL: features}, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn


def serving_input_fn():
    feature_placeholders = {
        TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
    }

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2])

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


def compute_errors(features, labels, predictions):
    labels = tf.expand_dims(labels, -1)  # rank 1 -> rank 2 to match rank of predictions

    if predictions.shape[1] == 1:
        loss = tf.losses.mean_squared_error(labels, predictions)
        rmse = tf.metrics.root_mean_squared_error(labels, predictions)
        return loss, rmse
    else:
        # one prediction for every input in sequence
        # get 1-N of (x + label)
        labelsN = tf.concat([features[TIMESERIES_COL], labels], axis=1)
        labelsN = labelsN[:, 1:]
        # loss is computed from the last 1/3 of the series
        N = (2 * N_INPUTS) // 3
        loss = tf.losses.mean_squared_error(labelsN[:, N:], predictions[:, N:])
        # rmse is computed from last prediction and last label
        lastPred = predictions[:, -1]
        rmse = tf.metrics.root_mean_squared_error(labels, lastPred)
        return loss, rmse

# RMSE when predicting same as last value
def same_as_last_benchmark(features, labels):
    predictions = features[TIMESERIES_COL][:,-1] # last value in input sequence
    return tf.metrics.root_mean_squared_error(labels, predictions)


# create the inference model
def sequence_regressor(features, labels, mode, params):
    # 1. run the appropriate model
    model_functions = {
        'linear': linear_model,
        'dnn': dnn_model,
        'cnn': cnn_model,
        'rnn': rnn_model,
        'rnn2': rnn2_model,
        'rnnN': rnnN_model}
    model_function = model_functions[params['model']]
    print(model_function)
    predictions = model_function(features, mode, params)

    # 2. loss function, training/eval ops
    loss = None
    train_op = None
    eval_metric_ops = None
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        loss, rmse = compute_errors(features, labels, predictions)
    
        if mode == tf.estimator.ModeKeys.TRAIN:
            # this is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                # 2b. set up training operation
                train_op = tf.contrib.layers.optimize_loss(
                    loss,
                    tf.train.get_global_step(),
                    learning_rate=params['learning_rate'],
                    optimizer="Adam")

        # 2c. eval metric
        eval_metric_ops = {
            "RMSE": rmse,
            "RMSE_same_as_last": same_as_last_benchmark(features, labels),
        }

    # 3. Create predictions
    if predictions.shape[1] != 1:
        predictions = predictions[:, -1]  # last predicted value
    predictions_dict = {"predicted": predictions}

    # 4. return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        export_outputs={
            'predictions': tf.estimator.export.PredictOutput(predictions_dict)}
    )


def train_and_evaluate(output_dir, hparams):
    get_train = read_dataset(hparams['train_data_path'],
                             tf.estimator.ModeKeys.TRAIN,
                             hparams['train_batch_size'])
    get_valid = read_dataset(hparams['eval_data_path'],
                             tf.estimator.ModeKeys.EVAL,
                             1000)
    estimator = tf.estimator.Estimator(model_fn=sequence_regressor,
                                       params=hparams,
                                       config=tf.estimator.RunConfig(
                                           save_checkpoints_steps=50,
                                         save_summary_steps=50
                                       ),
                                       model_dir=output_dir)
    train_spec = tf.estimator.TrainSpec(input_fn=get_train,
                                        max_steps=hparams['train_steps'])
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=get_valid,
                                      steps=None,
                                      exporters=exporter,
                                      #start_delay_secs=hparams['eval_delay_secs'],
                                      #throttle_secs=hparams['min_eval_frequency']
                                     )
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Writing /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py


In [11]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example implementation of code to run on the Cloud ML service.
"""

import traceback
import argparse
import json
import os
from . import model

if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--train_data_path',
      help='GCS or local path to training data',
      required=True
  )
  parser.add_argument(
      '--eval_data_path',
      help='GCS or local path to evaluation data',
      required=True
  )
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
      type=int,
      default=100
  )
  parser.add_argument(
      '--learning_rate',
      help='Initial learning rate for training',
      type=float,
      default=0.01
  )
  parser.add_argument(
      '--train_steps',
      help="""\
      Steps to run the training job for. A step is one batch-size,\
      """,
      type=int,
      default=0
  )
  parser.add_argument(
      '--sequence_length',
      help="""\
      This model works with fixed length sequences. 1-(N-1) are inputs, last is output
      """,
      type=int,
      default=10
  )
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  model_names = [name.replace('_model','') \
                   for name in dir(model) \
                     if name.endswith('_model')]
  parser.add_argument(
      '--model',
      help='Type of model. Supported types are {}'.format(model_names),
      required=True
  )
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud',
      default='junk'
  )
  parser.add_argument(
      '--eval_delay_secs',
      help='How long to wait before running first evaluation',
      default=10,
      type=int
  )
  parser.add_argument(
      '--min_eval_frequency',
      help='Minimum number of training steps between evaluations',
      default=60,
      type=int
  )

  args = parser.parse_args()
  hparams = args.__dict__
  
  # unused args provided by service
  hparams.pop('job_dir', None)
  hparams.pop('job-dir', None)

  output_dir = hparams.pop('output_dir')

  # Append trial_id to path if we are doing hptuning
  # This code can be removed if you are not using hyperparameter tuning
  output_dir = os.path.join(
      output_dir,
      json.loads(
          os.environ.get('TF_CONFIG', '{}')
      ).get('task', {}).get('trial', '')
  )

  # calculate train_steps if not provided
  if hparams['train_steps'] < 1:
     # 1,000 steps at batch_size of 100
     hparams['train_steps'] = (1000 * 100) // hparams['train_batch_size']
     print ("Training for {} steps".format(hparams['train_steps']))

  model.init(hparams)

  # Run the training job
  model.train_and_evaluate(output_dir, hparams)


Writing /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py


In [12]:
! pwd
#! ls
#!{PWD}/CNN_W2P_model/trainer

! ls /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer

/content/datalab/energy_forcasing/Sarah
__init__.py  model.py  task.py


## run the model locally on the vm

In [20]:
%%bash

DATADIR=$(pwd)/data/CNN_W2P_model 
OUTDIR=$(pwd)/CNN_W2P_model/trained
SEQ_LEN=50
rm -rf $OUTDIR
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/CNN_W2P_model/trainer \
   -- \
   --train_data_path="${DATADIR}/train-1.csv" \
   --eval_data_path="${DATADIR}/valid-1.csv"  \
   --output_dir=${OUTDIR} \
   --model=linear --train_steps=2000  --sequence_length=$SEQ_LEN

<function linear_model at 0x7f4fe01b7230>
<function linear_model at 0x7f4fe01b7230>
<function linear_model at 0x7f4fe01b7230>


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/train-1.csv', u'--eval_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/valid-1.csv', u'--output_dir=/content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trained', u'--model=linear', u'--train_steps=2000', u'--sequence_length=50'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spe

This is the newly created models location after it has been trained and it should be nameed a string of numbers, this time mine is named '1537798159/' whould yours should be a different string of numbers.

In [23]:
ls CNN_W2P_model/trained/export/exporter/

1537799028/


Feeding in a string of 49 prices to have the model preduct the 50th price. The data is not scaled down.

In [24]:
%%writefile test.json
[45.52, 45.24, 43.792, 24.0, 37.488, 31.400000000000002, 38.800000000000004, 34.792, 47.656000000000006, 32.104000000000006, 36.0, 39.88, 40.056000000000004, 39.2, 34.952, 50.42400000000001, 36.472, 42.848000000000006, 30.496, 44.712, 32.128, 52.08, 31.664, 47.88, 44.0, 51.352000000000004, 42.624, 54.879999999999995, 32.800000000000004, 31.400000000000002, 30.680000000000003, 44.608000000000004, 32.256, 50.072, 37.424, 39.88, 47.752, 44.800000000000004, 52.08, 46.952, 31.6, 24.0, 52.08, 45.760000000000005, 24.0, 40.608000000000004, 37.664, 36.080000000000005, 11.872]

Overwriting test.json


In [25]:
%bash
model=$(ls CNN_W2P_model/trained/export/exporter | tail -1)
gcloud ml-engine local predict \
    --model-dir=CNN_W2P_model/trained/export/exporter/$model \
    --json-instances=./test.json

PREDICTED
[27.66790771484375]


  from ._conv import register_converters as _register_converters
2018-09-24 14:24:48.221923: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA



The 'predicted' value above should be in stark contrast to the value you will get out when you feed in a string of 0's (as demonstrated below).

In [26]:
%%writefile test.json
{"price": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}

Overwriting test.json


In [27]:
%bash
model=$(ls CNN_W2P_model/trained/export/exporter | tail -1)
gcloud ml-engine local predict \
    --model-dir=CNN_W2P_model/trained/export/exporter/$model \
    --json-instances=./test.json

PREDICTED
[0.3067505359649658]


  from ._conv import register_converters as _register_converters
2018-09-24 14:24:55.554682: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA

